In [2]:
import cv2
import argparse
import imutils
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from imutils.video import VideoStream

In [3]:
MODEL_DIR = '../../model/'
IMG_HEIGHT = 256
IMG_WIDTH = 256
CLASS_NAMES = ['correctly-masked', 'not-masked', 'uncorrectly-masked']

In [4]:
model = keras.models.load_model(MODEL_DIR)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)       

In [5]:
def get_face(img) -> Image.Image: 
    crop_img = img[y:y+h, x:x+w]
    cv2.imshow("cropped", crop_img)
    # Convert image from GBR to RGB
    rgb_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(rgb_img)
    newsize = (IMG_WIDTH, IMG_HEIGHT)
    resized_img = pil_img.resize(newsize)
    return resized_img

def predict(resized_img):
    img_array = keras.preprocessing.image.img_to_array(resized_img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    prediction = CLASS_NAMES[np.argmax(score)]
    confidence = 100 * np.max(score)
    return prediction, confidence
    
def get_color(prediction) -> tuple:
    green = (36,255,12)
    if prediction == CLASS_NAMES[0]:
        color = green
    elif prediction == CLASS_NAMES[1]:
        color = (0, 255, 0)
    else:
        color = (0, 0, 255)

In [6]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
print("[INFO] starting video stream...")
cap = cv2.VideoCapture(0)
_, img = cap.read()

if (cap.isOpened() == False):
    print("Unable to read camera feed")

while True:
    # Read the frame
    _, img = cap.read()
    # Convert to grayscale (haarcascade works with grayscale)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect the faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        resized_img = get_face(img)
        prediction, confidence= predict(resized_img)
        color = get_color(prediction)
        print(prediction)
        cv2.rectangle(img, (x, y), (x+w, y+h), (36,255,12), 2)
        cv2.putText(img, str(confidence), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        
    # Display
    cv2.imshow('img', img)
    
    # Stop if escape key is pressed
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
exit()

[INFO] starting video stream...
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
not-masked
